In [1]:
import os
os.chdir("../")

In [2]:
import numpy as np
import torch

In [3]:
from storage.har_datasets import HARTHDataset, UCI_HARDataset, sts_medoids, StreamingTimeSeriesCopy, split_by_test_subject
from s3ts.api.dms.har_datasets import LDFDataset, DFDataset
from storage.label_mappings import *
from s3ts.api.nets.methods import create_model_from_DM, train_model, test_model

In [4]:
from torchvision.transforms import Normalize

In [5]:
HARTH_LABELS

{1: 'walking',
 2: 'running',
 3: 'shuffling',
 4: 'stairs_up',
 5: 'stairs_down',
 6: 'standing',
 7: 'sitting',
 8: 'lying',
 13: 'cycking_sit',
 14: 'cycling_stand',
 130: 'cycling_sit_idle',
 140: 'cycling_stand_idle'}

In [6]:
label_mapping = np.zeros(141)
label_mapping[1:9] = np.arange(8)
label_mapping[13] = 8
label_mapping[14] = 9
label_mapping[130] = 10
label_mapping[140] = 11

ds = HARTHDataset("./datasets/HARTH/", wsize=48, normalize=True, label_mapping=label_mapping)

In [7]:
len(ds)

6225494

In [8]:
if not os.path.exists("./datasets/HARTH/meds.npz"):
    meds = sts_medoids(ds, n=500)
    with open("./datasets/HARTH/meds.npz", "wb") as f:
        np.save(f, meds)
else:
    meds = np.load("./datasets/HARTH/meds.npz")

In [9]:
dfds = DFDataset(ds, patterns=meds, w=0.1, dm_transform=None, ram=False)

Loading cached dissimilarity frames if available...


In [10]:
DM = []

np.random.seed(42)
for i in np.random.choice(np.arange(len(dfds)), 500):
    dm, _, _ = dfds[i]
    DM.append(dm)

DM = torch.stack(DM)

dm_transform = Normalize(mean=DM.mean(dim=[0, 2, 3]), std=DM.std(dim=[0, 2, 3]))

In [11]:
dfds.dm_transform = dm_transform

In [12]:
data_split = split_by_test_subject(ds, 21)

dm = LDFDataset(dfds, data_split=data_split, batch_size=128, random_seed=42, num_workers=8)

In [13]:
model = create_model_from_DM(dm, name=None, 
        dsrc="img", arch="cnn", task="cls")

Input shape:  torch.Size([1, 12, 48, 48])
Latent shape:  torch.Size([1, 40, 3, 48])


In [ ]:
model, data = train_model(dm, model, max_epochs=2)
print(data)